In [ ]:
%%capture
# Install dependencies for ragas
!pip install -q langchain sentence-transformers datasets langchain-google-genai ragas unstructured

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [ ]:
from tqdm.notebook import tqdm
import os
import pandas as pd
from typing import Optional, List, Tuple
import json

from datasets import Dataset
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    answer_correctness,
    context_recall,
    context_precision,
    context_entity_recall
)
from ragas import evaluate

pd.set_option("display.max_colwidth", None)

In [ ]:
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

## Evaluate with `ragas`

### Init LLM and Embedding for `ragas`

In [ ]:
# https://python.langchain.com/docs/integrations/llms/google_ai/
from langchain_google_genai import GoogleGenerativeAI

langchain_llm = GoogleGenerativeAI(model="gemini-pro", google_api_key=GOOGLE_API_KEY)

In [ ]:

# https://python.langchain.com/docs/integrations/text_embedding/bge_huggingface/
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-m3"
model_kwargs = {"device": device} # Use CPU to avoid RateLimitExceed error in testset generation
encode_kwargs = {"normalize_embeddings": True}
embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

### Load benchmarking dataset

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
benchmark_dataset = pd.read_csv("gdrive/MyDrive/LLM01 - RắnGPT/Code/Benchmark QA/benchmark_test_set.csv")
benchmark_dataset['ground_truth'] = benchmark_dataset['ground_truths']
# benchmark_dataset['context'] = benchmark_dataset['contexts']
benchmark_dataset = benchmark_dataset[['question', 'ground_truth', 'contexts']]

for i in range(len(benchmark_dataset)):
    benchmark_dataset['contexts'][i] = [benchmark_dataset['contexts'][i]]

benchmark_dataset.head()

In [ ]:
len(benchmark_dataset)

58

### Evaluate embedding module

#### Evaluate baseline

##### Retrieve contexts

In [ ]:
%%capture
# Install dependencies for embedding module
!pip install -q qdrant-client FlagEmbedding

In [ ]:
%%capture
# Lots of download outputs, show use capture to hide them
from FlagEmbedding import BGEM3FlagModel
baseline_SENTEMB = BGEM3FlagModel('BAAI/bge-m3',
                       use_fp16=True, device=device) # Setting use_fp16 to True speeds up computation with a slight performance degradation

In [ ]:
from qdrant_client import QdrantClient, models
# 'https://e69451a1-5421-44b2-804e-7cfd44b35d4f.us-east4-0.gcp.cloud.qdrant.io:6333'
client = QdrantClient(
  url = "https://e69451a1-5421-44b2-804e-7cfd44b35d4f.us-east4-0.gcp.cloud.qdrant.io:6333",
  api_key="SuRvESqZ_mYMjKmce8oZa1AgZdVwn8MUs1F5KiVnO7bBhPEwFAwCDQ",
)


In [ ]:
collection_name = "rangpt"
top_k_retrieve = 10

prompt = "rắn cạp nong ăn gì"

query_results = client.search(
    collection_name=collection_name,
    query_vector=SENTEMB.encode(prompt)['dense_vecs'].tolist(),
    limit=top_k_retrieve,
)

raw_contexts = []
for query_result in query_results:
    raw_contexts.append(query_result.payload['text'])
    print(query_result.payload['text'], "score:", query_result.score)
    print()

Tên Việt Nam: Đẻn cạp nong 
 Tên Latinh / Tên khoa học: Hydrophis atriceps Günther 
 Thức ăn: Chủ yếu là cá (Karthikeyan & Balasubramaniant, 2007). score: 0.63664424

Tên Việt Nam: Đẻn cạp nong môi vàng 
 Tên Latinh / Tên khoa học: Laticauda colubrina 
 Thức ăn: Chủ yếu là lươn (Heatwole và cs., 2012). score: 0.6181069

Tên Việt Nam: RẮN CẠP NONG ĐẦU ĐỎ 
 Tên Latinh / Tên khoa học: Bungarus flaviceps 
 Thức ăn: Thức ăn chủ yếu của loài này là các loài rắn khác và loài thằn lằn bóng chân ngắn score: 0.61097944

Tên Việt Nam: RẮN CẠP NONG 
 Tên Latinh / Tên khoa học: Bungarus fasciatus 
 Môi trường sống: Là loài rắn độc phổ biến khắp nơi, một trong những loài rắn độc phổ biến nhất ở đồng bằng, trung du và miền núi.  Sống trong rừng hoặc những nơi gần chỗ ở của con người, thường gặp chúng nhiều hơn cả ở những địa hình cao giáp với nước, sống trong hang chuột hay hang mối đã bỏ ở bờ ruộng, gò đống, bờ sông, bờ đê, vườn tược, bụi tre, bờ ao.  Trong mùa khô lạnh chúng thường ẩn náu đơn độc, 

##### Rerank retrieved contexts

In [ ]:
%%capture
from FlagEmbedding import FlagReranker
reranker = FlagReranker('BAAI/bge-reranker-v2-m3', use_fp16=False) # Setting use_fp16 to True speeds up computation with a slight performance degradation

In [ ]:
import numpy as np

rerank_scores = reranker.compute_score([[prompt, raw_contexts[i]] for i in range(len(raw_contexts))], normalize=True)
rerank_scores = np.array(rerank_scores)
print("Rerank score", rerank_scores)

top_k_rerank_indices = rerank_scores.argsort()[-len(raw_contexts):][::-1]

top_k_rerank = 5
reranked_contexts = [raw_contexts[top_k_rerank_indices[i]] for i in range(len(raw_contexts))][:top_k_rerank]

Rerank score [0.93644386 0.89783206 0.98505699 0.55763013 0.31441234 0.54628216
 0.97976084 0.68679762 0.07675879 0.32336926]


In [ ]:
print(f"QUESTION: {prompt}")
print("="*50)
for i in range(5):
  print()
  print(f"ANSWER {i+1} - score: {rerank_scores[top_k_rerank_indices[i]]} \n {reranked_contexts[i]}")
  print()
  print("="*50)

QUESTION: rắn cạp nong ăn gì

ANSWER 1 - score: 0.9850569858235351 
 Tên Việt Nam: RẮN CẠP NONG ĐẦU ĐỎ 
 Tên Latinh / Tên khoa học: Bungarus flaviceps 
 Thức ăn: Thức ăn chủ yếu của loài này là các loài rắn khác và loài thằn lằn bóng chân ngắn


ANSWER 2 - score: 0.979760841132862 
 Tên Việt Nam: RẮN CẠP NONG 
 Tên Latinh / Tên khoa học: Bungarus fasciatus 
 Thức ăn:  Chúng kiếm ăn về ban đêm, bắt các loài rắn khác, đôi khi ăn cả thằn lằn


ANSWER 3 - score: 0.9364438619568783 
 Tên Việt Nam: Đẻn cạp nong 
 Tên Latinh / Tên khoa học: Hydrophis atriceps Günther 
 Thức ăn: Chủ yếu là cá (Karthikeyan & Balasubramaniant, 2007).


ANSWER 4 - score: 0.8978320609927664 
 Tên Việt Nam: Đẻn cạp nong môi vàng 
 Tên Latinh / Tên khoa học: Laticauda colubrina 
 Thức ăn: Chủ yếu là lươn (Heatwole và cs., 2012).


ANSWER 5 - score: 0.6867976210397213 
 Tên Việt Nam: RẮN CẠP NIA NAM 
 Tên Latinh / Tên khoa học: Bungarus candidus 
 Thức ăn:  Thức ăn chủ yếu là các loại rắn khác, đôi khi chúng ăn cả th

In [ ]:
def retrieve_context(prompt, client, collection_name, top_k_retrieve=10, top_k_rerank=5, rerank=True):

  query_results = client.search(
      collection_name=collection_name,
      query_vector=SENTEMB.encode(prompt)['dense_vecs'].tolist(),
      limit=top_k_retrieve,
  )

  raw_contexts = []
  for query_result in query_results:
      raw_contexts.append(query_result.payload['text'])

  if rerank:
    rerank_scores = reranker.compute_score([[prompt, raw_contexts[i]] for i in range(len(raw_contexts))], normalize=True)
    rerank_scores = np.array(rerank_scores)

    top_k_rerank_indices = rerank_scores.argsort()[-len(raw_contexts):][::-1]

    reranked_contexts = [raw_contexts[top_k_rerank_indices[i]] for i in range(len(raw_contexts))]

    return reranked_contexts[:top_k_rerank]

In [ ]:
retrieve_context("Cần làm gì khi bị rắn cắn?", client, collection_name="rangpt")

['Những bước đầu tiên cần làm khi bị rắn cắn là gì? \n • Giữ bình tĩnh và không hoảng sợ.\n• Gỡ bỏ mọi đồ trang sức, vòng tay hoặc quần áo bó sát khỏi vùng bị cắn vì khi vết thương sưng có thể làm thắt chặt gây tổn thương mô.\n• Rửa sạch vết cắn bằng nước sạch\n• Bất động vùng bị cắn bằng nẹp hoặc băng.\n• Hạ thấp vùng bị cắn hơn so với tim.\n• Không rạch vết cắn hoặc cố gắng hút nọc độc.\n• Không cho nạn nhân bất cứ thức uống gì ngoài nước lọc.\n• Theo dõi các triệu chứng của nạn nhân và đưa họ đến bệnh viện hoặc cơ sở y tế gần nhất ngay lập tức.',
 'Các triệu chứng của người bị rắn độc cắn là gì? \n Các triệu chứng của vết rắn cắn có nọc độc bao gồm:\nTại vết cắn: - Sưng, đau hoặc kích thước của vết cắn tăng lên nhanh chóng. - Đau nhức hoặc cảm giác nóng rát quanh vùng vết cắn. - Có thể xuất hiện các dấu hiệu khác như: bầm tím, chảy máu, phồng rộp, hoại tử da.\nTriệu chứng toàn thân: - Buồn nôn, nôn mửa. - Tiêu chảy. - Chóng mặt, hoa mắt. - Đau đầu. - Mệt mỏi, yếu ớt. - Khó thở. - Lo

##### Run Retriever on benchmarking dataset

In [ ]:
retrieved_contexts = []

for question in tqdm(benchmark_dataset['question']):
  retrieved_contexts.append(retrieve_context(question, client, collection_name="rangpt"))

  0%|          | 0/58 [00:00<?, ?it/s]

In [ ]:
benchmark_dataset['contexts'] = retrieved_contexts

##### Evaluation




In [ ]:
# dataset = Dataset.from_dict(data_samples)
dataset = Dataset.from_pandas(benchmark_dataset)

score = evaluate(dataset, metrics=[context_precision, context_recall, context_entity_recall], llm=langchain_llm, embeddings=embeddings)
retriever_score = score.to_pandas()

Evaluating:   0%|          | 0/116 [00:00<?, ?it/s]

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
}
metadata {
  key: "quota_limit"
  value: "GenerateContentRequestsPerMinutePerProjectPerRegion"
}
metadata {
  key: "quota_limit_value"
  value: "60"
}
metadata {
  key: "quota_location"
  value: "us-west4"
}
metadata {
  key: "quota_metric"
  value: "generativelanguage.googleapis.com/generate_content_requests"
}
metadata {
  key: "service"
  value: "generativelanguage.googleapis.com"
}
, links {
  description: "Request a higher quota limit."
  url: "https://cloud.google.com/docs/quota#requesting_higher_quota"
}
].
domain: "googleapis.com"
metadata {
  key: "consumer"
  value: "projects/813003764678"
}
metadata {
  key: "quota_limit"
  value: "GenerateContentRequestsPerMinutePerProjectPerRegion"
}
metadata {
  key: "quota_limit_value"
  value: "60"
}
metadata {
  key: "quota_location"
  value: "us-west4"
}
metadata {
  key: "quota_metric"
  value: "generativelanguage.googleapis.com/generate_content_requests"
}
metadata {
  key: 

In [ ]:
retriever_score.head()

question  \
0           Tên khoa học của rắn lục đuôi đỏ mắt nhỏ là gì?   
1                                 Rắn lột da bằng cách nào?   
2  Rắn lục đầu trắng phân bố ở những tỉnh nào của Việt Nam?   
3         Loài rắn biển này có bao nhiêu hàng vảy quanh cổ?   
4          Rắn sọc dưa có习 tính săn đuổi mồi chủ yếu là gì?   

                                            ground_truth  \
0                                  Cryptelytrops macrops   
1  Rắn lột da bằng cách lộn ngược cơ thể từ bên trong ra   
2                 Cao Bằng, Bắc Kạn, Vĩnh Phúc, Lạng Sơn   
3                                             31–43 hàng   
4            Chuột, ngoài ra có cả thằn lằn và ếch nhái.   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

In [ ]:
retriever_score.to_csv("gdrive/MyDrive/LLM01 - RắnGPT/Code/Benchmark QA/retriever_evaluation.csv", index=False)

##### Analyze evaluation score

In [ ]:
retriever_score = pd.read_csv("gdrive/MyDrive/LLM01 - RắnGPT/Code/Benchmark QA/retriever_evaluation.csv")

In [ ]:
# benchmark_dataset = pd.read_csv("gdrive/MyDrive/LLM01 - RắnGPT/Code/Benchmark QA/retriever_evaluation.csv")
# benchmark_dataset = benchmark_dataset[['question', 'ground_truth', 'contexts']]

# for i in range(len(benchmark_dataset)):
#     benchmark_dataset.loc[i, "contexts"] = eval(benchmark_dataset.loc[i, "contexts"].replace("\n", ","))

# dataset = Dataset.from_pandas(benchmark_dataset)

# score = evaluate(dataset, metrics=[context_entity_recall], llm=langchain_llm, embeddings=embeddings)
# retriever_score = score.to_pandas()
# # 0.3298029537162835

In [ ]:
print("Context precision", retriever_score['context_precision'].mean())
print("Context_precision - Measures how relevant the retrieved context is to the question, conveying the quality of the retrieval pipeline")
print("https://docs.ragas.io/en/stable/concepts/metrics/context_precision.html")
print()
print("Context recal", retriever_score['context_recall'].mean())
print("Context_recall - Measures the retriever’s ability to retrieve all necessary information required to answer the question.")
print("https://docs.ragas.io/en/stable/concepts/metrics/context_recall.html")
print()
print("Context entities recall", retriever_score['context_entity_recall'].mean())
print("Context entities recall - Measure what fraction of entities are recalled from ground_truths. This metric is useful in fact-based use cases like tourism help desk, historical QA, etc.")
print("https://docs.ragas.io/en/stable/concepts/metrics/context_entities_recall.html")

Context precision 0.914692982398044
Context_precision - Measures how relevant the retrieved context is to the question, conveying the quality of the retrieval pipeline
https://docs.ragas.io/en/stable/concepts/metrics/context_precision.html

Context recal 0.9166666666666666
Context_recall - Measures the retriever’s ability to retrieve all necessary information required to answer the question.
https://docs.ragas.io/en/stable/concepts/metrics/context_recall.html

Context entities recall 0.4536124772594997
Context entities recall - Measure what fraction of entities are recalled from ground_truths. This metric is useful in fact-based use cases like tourism help desk, historical QA, etc.
https://docs.ragas.io/en/stable/concepts/metrics/context_entities_recall.html


#### Evaluate finetuned

##### Retrieve contexts

In [ ]:
%%capture
# Install dependencies for embedding module
!pip install -q qdrant-client FlagEmbedding

In [ ]:
%%capture
# Lots of download outputs, show use capture to hide them
from FlagEmbedding import BGEM3FlagModel
SENTEMB = BGEM3FlagModel('salforis/finetune_bge-m3_ran_data',
                       use_fp16=True, device=device) # Setting use_fp16 to True speeds up computation with a slight performance degradation

In [ ]:
from qdrant_client import QdrantClient, models
# 'https://e69451a1-5421-44b2-804e-7cfd44b35d4f.us-east4-0.gcp.cloud.qdrant.io:6333'
client = QdrantClient(
  url = "https://e69451a1-5421-44b2-804e-7cfd44b35d4f.us-east4-0.gcp.cloud.qdrant.io:6333",
  api_key="SuRvESqZ_mYMjKmce8oZa1AgZdVwn8MUs1F5KiVnO7bBhPEwFAwCDQ",
)


In [ ]:
https://www.kaggle.com/code/panda1835/rangpt-inference/edit

In [ ]:
collection_name = "rangpt-bge-sft"
top_k_retrieve = 10

prompt = "Rắn biển bụng trắng có bao nhiêu hàng vảy quanh thân?"

query_results = client.search(
    collection_name=collection_name,
    query_vector=SENTEMB.encode(prompt)['dense_vecs'].tolist(),
    limit=top_k_retrieve,
)

raw_contexts = []
for query_result in query_results:
    raw_contexts.append(query_result.payload['text'])
    print(query_result.payload['text'], "score:", query_result.score)
    print()

Tên Việt Nam: Đẻn đuôi vàng, đẻn sọc dưa 
 Tên Latinh / Tên khoa học: Hydrophis platura 
 Đặc điểm nhận dạng (màu sắc cơ thể): Hàng vảy quanh thân 49–67; vảy bụng 264–406 có thể bị nứt vỡ hoặc không và tương tự với các vảy đường bên; răng xương hàm trên phía sau răng nanh độc 7–11.
  Màu sắc: cho đến nay, đây là loài rắn biển duy nhất có màu đen phía trên lưng, phần bụng có màu vàng. score: 0.40402284

Tên Việt Nam: Rắn lục biển 
 Tên Latinh / Tên khoa học: Hydrophis viperina 
 Đặc điểm nhận dạng (màu sắc cơ thể): Dễ dàng xác định bỡi vảy bụng của chúng, vảy bụng rộng ở phía trước và nhỏ dần ở phía sau. Hàng vảy quanh cổ 27–34; hàng vảy quanh thân 37–50; vảy bụng 226–274; răng xương hàm trên phía sau răng nanh độc 5.
  Màu sắc: cơ thể có màu xám ở phần trên lưng, phần bụng có màu trắng, không có khoanh score: 0.3988151

Tên Việt Nam: Đẻn nhiều răng 
 Tên Latinh / Tên khoa học: Hydrophis caerulescens 
 Đặc điểm nhận dạng (màu sắc cơ thể): Là loài rắn biển duy nhất có hơn 13 răng của xươ

##### Rerank retrieved contexts

In [ ]:
%%capture
from FlagEmbedding import FlagReranker
reranker = FlagReranker('BAAI/bge-reranker-v2-m3', use_fp16=False) # Setting use_fp16 to True speeds up computation with a slight performance degradation

In [ ]:
import numpy as np

rerank_scores = reranker.compute_score([[prompt, raw_contexts[i]] for i in range(len(raw_contexts))], normalize=True)
rerank_scores = np.array(rerank_scores)
print("Rerank score", rerank_scores)

top_k_rerank_indices = rerank_scores.argsort()[-len(raw_contexts):][::-1]

top_k_rerank = 5
reranked_contexts = [raw_contexts[top_k_rerank_indices[i]] for i in range(len(raw_contexts))][:top_k_rerank]

Rerank score [0.93644392 0.89783243 0.98505702 0.3539522  0.97976088 0.68679831
 0.32336972 0.62433862 0.17708549 0.13130312]


In [ ]:
print(f"QUESTION: {prompt}")
print("="*50)
for i in range(5):
  print()
  print(f"ANSWER {i+1} - score: {rerank_scores[top_k_rerank_indices[i]]} \n {reranked_contexts[i]}")
  print()
  print("="*50)

QUESTION: rắn cạp nong ăn gì

ANSWER 1 - score: 0.9850570209180429 
 Tên Việt Nam: RẮN CẠP NONG ĐẦU ĐỎ 
 Tên Latinh / Tên khoa học: Bungarus flaviceps 
 Thức ăn: Thức ăn chủ yếu của loài này là các loài rắn khác và loài thằn lằn bóng chân ngắn


ANSWER 2 - score: 0.979760883682385 
 Tên Việt Nam: RẮN CẠP NONG 
 Tên Latinh / Tên khoa học: Bungarus fasciatus 
 Thức ăn:  Chúng kiếm ăn về ban đêm, bắt các loài rắn khác, đôi khi ăn cả thằn lằn


ANSWER 3 - score: 0.9364439187164557 
 Tên Việt Nam: Đẻn cạp nong 
 Tên Latinh / Tên khoa học: Hydrophis atriceps Günther 
 Thức ăn: Chủ yếu là cá (Karthikeyan & Balasubramaniant, 2007).


ANSWER 4 - score: 0.8978324327830697 
 Tên Việt Nam: Đẻn cạp nong môi vàng 
 Tên Latinh / Tên khoa học: Laticauda colubrina 
 Thức ăn: Chủ yếu là lươn (Heatwole và cs., 2012).


ANSWER 5 - score: 0.6867983133924961 
 Tên Việt Nam: RẮN CẠP NIA NAM 
 Tên Latinh / Tên khoa học: Bungarus candidus 
 Thức ăn:  Thức ăn chủ yếu là các loại rắn khác, đôi khi chúng ăn cả th

In [ ]:
def retrieve_context(prompt, client, collection_name, top_k_retrieve=10, top_k_rerank=5, rerank=True):

  query_results = client.search(
      collection_name=collection_name,
      query_vector=SENTEMB.encode(prompt)['dense_vecs'].tolist(),
      limit=top_k_retrieve,
  )

  raw_contexts = []
  for query_result in query_results:
      raw_contexts.append(query_result.payload['text'])

  if rerank:
    rerank_scores = reranker.compute_score([[prompt, raw_contexts[i]] for i in range(len(raw_contexts))], normalize=True)
    rerank_scores = np.array(rerank_scores)

    top_k_rerank_indices = rerank_scores.argsort()[-len(raw_contexts):][::-1]

    reranked_contexts = [raw_contexts[top_k_rerank_indices[i]] for i in range(len(raw_contexts))]

    return reranked_contexts[:top_k_rerank]

In [ ]:
retrieve_context("Cần làm gì khi bị rắn cắn?", client, collection_name=collection_name)

['Những bước đầu tiên cần làm khi bị rắn cắn là gì? \n • Giữ bình tĩnh và không hoảng sợ.\n• Gỡ bỏ mọi đồ trang sức, vòng tay hoặc quần áo bó sát khỏi vùng bị cắn vì khi vết thương sưng có thể làm thắt chặt gây tổn thương mô.\n• Rửa sạch vết cắn bằng nước sạch\n• Bất động vùng bị cắn bằng nẹp hoặc băng.\n• Hạ thấp vùng bị cắn hơn so với tim.\n• Không rạch vết cắn hoặc cố gắng hút nọc độc.\n• Không cho nạn nhân bất cứ thức uống gì ngoài nước lọc.\n• Theo dõi các triệu chứng của nạn nhân và đưa họ đến bệnh viện hoặc cơ sở y tế gần nhất ngay lập tức.',
 'Các triệu chứng của người bị rắn độc cắn là gì? \n Các triệu chứng của vết rắn cắn có nọc độc bao gồm:\nTại vết cắn: - Sưng, đau hoặc kích thước của vết cắn tăng lên nhanh chóng. - Đau nhức hoặc cảm giác nóng rát quanh vùng vết cắn. - Có thể xuất hiện các dấu hiệu khác như: bầm tím, chảy máu, phồng rộp, hoại tử da.\nTriệu chứng toàn thân: - Buồn nôn, nôn mửa. - Tiêu chảy. - Chóng mặt, hoa mắt. - Đau đầu. - Mệt mỏi, yếu ớt. - Khó thở. - Lo

##### Run Retriever on benchmarking dataset

In [ ]:
retrieved_contexts = []

for question in tqdm(benchmark_dataset['question']):
  retrieved_contexts.append(retrieve_context(question, client, collection_name=collection_name))

  0%|          | 0/58 [00:00<?, ?it/s]

In [ ]:
benchmark_dataset['contexts'] = retrieved_contexts

##### Evaluation




In [ ]:
# dataset = Dataset.from_dict(data_samples)
dataset = Dataset.from_pandas(benchmark_dataset)

score = evaluate(dataset, metrics=[context_precision, context_recall, context_entity_recall], llm=langchain_llm, embeddings=embeddings, raise_exceptions=False)
retriever_score = score.to_pandas()

In [ ]:
dataset.to_pandas()

question  \
0                                   Tên khoa học của rắn lục đuôi đỏ mắt nhỏ là gì?   
1                                                         Rắn lột da bằng cách nào?   
2                          Rắn lục đầu trắng phân bố ở những tỉnh nào của Việt Nam?   
3                                 Loài rắn biển này có bao nhiêu hàng vảy quanh cổ?   
4                                  Rắn sọc dưa có习 tính săn đuổi mồi chủ yếu là gì?   
5                             Rắn biển bụng trắng có bao nhiêu hàng vảy quanh thân?   
6                                                     Đẻn mỏ sinh con hay đẻ trứng?   
7                                   Rắn đầu gai có đặc điểm nhận dạng nào đặc biệt?   
8                                   Vật nuôi phải được đảm bảo có nguồn gốc từ đâu?   
9                                    Rắn lục biển có bao nhiêu hàng vảy quanh thân?   
10       Rắn đẻn khoanh mờ có bao nhiêu răng xương hàm trên phía sau răng nanh độc?   
11               Ở môi trường nào loài rắn biển Hydrophis stokeii thường xuất hiện?   
12                                        Rắn hổ xiên mắt thường sống ở độ cao nào?   
13  Loài rắn biển Emydocephalus annulatus thường sống ở độ sâu tối đa là bao nhiêu?   
14                            Rắn sãi boulenger phân bố ở khu vực nào của Việt Nam?   
15                                Tên Latinh / Tên khoa học của loài rắn này là gì?   
16                                       Rắn hổ chúa đẻ khoảng bao nhiêu trứng/lứa?   
17                           Loài rắn này phân bố ở những khu vực nào tại Việt Nam?   
18                                    Rắn hoa cân vân nam phân bố ở đâu trong nước?   
19                           Rắn rào ngọc phân bố ở những khu vực nào tại Việt Nam?   
20                       Rắn bố mẹ có chăm sóc con của mình sau khi chúng nở không?   
21                                   Rắn roi mũi thường thò thụt lưỡi ra để làm gì?   
22                                                Rắn hổ mây gờ chủ yếu ăn loài gì?   
23                                                 Loài rắn này có nguồn gốc ở đâu?   
24                                    Loài rắn hổ mây di linh được phát hiện ở đâu?   
25                                                      Rắn có sống theo bầy không?   
26                               Rắn hổ mang một mắt kính dài bao nhiêu khi mới nở?   
27                                         Loại rắn nào có thể đẻ 4 - 7 trứng thai?   
28                                 Rắn mai gầm bắc phân bố ở tỉnh nào của Việt Nam?   
29                                              Rắn lục mũi hếch thường sống ở đâu?   
30                                     Loài rắn này mới được phát hiện vào năm nào?   
31                                             Rắn hồng răng lớn sống ở độ cao nào?   
32                                     Độc tố học của Hydrophis cyanocinctus là gì?   
33                         Nơi phân bố của loài rắn Sãi kut kai tại Việt Nam ở đâu?   
34                           Nhiệt độ ưa thích của mỗi loài rắn có khác nhau không?   
35                                       Loài rắn này thường xuất hiện vào mùa nào?   
36                                       Rắn biển sọc có khoảng bao nhiêu vảy bụng?   
37                                                Rắn lục đuôi đỏ thuộc họ rắn nào?   
38       Ở Mỹ, loài rắn nào xâm lấn gây thiệt hại đáng kể cho cá sấu, hươu và chim?   
39                      Rắn đẻn cơm có bao nhiêu hàng vảy bên lớn hơn các vảy khác?   
40                                   Thức ăn chính của loài Rắn rồng đầu đen là gì?   
41                           Rắn rào xiêm phân bố ở những khu vực nào của Việt Nam?   
42                             Rắn sãi mép trắng phân bố ở nước nào ngoài Việt Nam?   
43                                     Rắn ráo thường phân bố ở những quốc gia nào?   
44                                                           Rắn nước chuyên ăn gì?   
45                                                    Rắn dùng khứu giác 

In [ ]:
retriever_score.head(10)

question  \
0           Tên khoa học của rắn lục đuôi đỏ mắt nhỏ là gì?   
1                                 Rắn lột da bằng cách nào?   
2  Rắn lục đầu trắng phân bố ở những tỉnh nào của Việt Nam?   
3         Loài rắn biển này có bao nhiêu hàng vảy quanh cổ?   
4          Rắn sọc dưa có习 tính săn đuổi mồi chủ yếu là gì?   
5     Rắn biển bụng trắng có bao nhiêu hàng vảy quanh thân?   
6                             Đẻn mỏ sinh con hay đẻ trứng?   
7           Rắn đầu gai có đặc điểm nhận dạng nào đặc biệt?   
8           Vật nuôi phải được đảm bảo có nguồn gốc từ đâu?   
9            Rắn lục biển có bao nhiêu hàng vảy quanh thân?   

                                                                                                                                       ground_truth  \
0                                                                                                                             Cryptelytrops macrops   
1                                                                                             Rắn lột da bằng cách lộn ngược cơ thể từ bên trong ra   
2                                                                                                            Cao Bằng, Bắc Kạn, Vĩnh Phúc, Lạng Sơn   
3                                                                                                                                        31–43 hàng   
4                                                                                                       Chuột, ngoài ra có cả thằn lằn và ếch nhái.   
5                                                                                                                                        39–45 hàng   
6                                                                                                                                  Đẻn mỏ sinh con.   
7  Rắn đầu gai có đặc điểm nhận dạng duy nhất là có gai trên đầu, cụ thể là các vảy trước mắt, sau mắt, trên mắt, dưới mắt có các gai phía sau vảy.   
8                                                                                                                      Môi trường sinh sản nhân tạo   
9                                                                                                                                             37–50   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

In [ ]:
retriever_score.to_csv("gdrive/MyDrive/LLM01 - RắnGPT/Code/Benchmark QA/ft_retriever_evaluation.csv", index=False)

##### Analyze evaluation score

In [ ]:
retriever_score = pd.read_csv("gdrive/MyDrive/LLM01 - RắnGPT/Code/Benchmark QA/ft_retriever_evaluation.csv")

In [ ]:
# benchmark_dataset = pd.read_csv("gdrive/MyDrive/LLM01 - RắnGPT/Code/Benchmark QA/retriever_evaluation.csv")
# benchmark_dataset = benchmark_dataset[['question', 'ground_truth', 'contexts']]

# for i in range(len(benchmark_dataset)):
#     benchmark_dataset.loc[i, "contexts"] = eval(benchmark_dataset.loc[i, "contexts"].replace("\n", ","))

# dataset = Dataset.from_pandas(benchmark_dataset)

# score = evaluate(dataset, metrics=[context_entity_recall], llm=langchain_llm, embeddings=embeddings)
# retriever_score = score.to_pandas()
# # 0.3298029537162835

In [ ]:
print("Context precision", retriever_score['context_precision'].mean())
print("Context_precision - Measures how relevant the retrieved context is to the question, conveying the quality of the retrieval pipeline")
print("https://docs.ragas.io/en/stable/concepts/metrics/context_precision.html")
print()
print("Context recal", retriever_score['context_recall'].mean())
print("Context_recall - Measures the retriever’s ability to retrieve all necessary information required to answer the question.")
print("https://docs.ragas.io/en/stable/concepts/metrics/context_recall.html")
print()
print("Context entities recall", retriever_score['context_entity_recall'].mean())
print("Context entities recall - Measure what fraction of entities are recalled from ground_truths. This metric is useful in fact-based use cases like tourism help desk, historical QA, etc.")
print("https://docs.ragas.io/en/stable/concepts/metrics/context_entities_recall.html")

Context precision 0.856226053584304
Context_precision - Measures how relevant the retrieved context is to the question, conveying the quality of the retrieval pipeline
https://docs.ragas.io/en/stable/concepts/metrics/context_precision.html

Context recal 0.8596491228070176
Context_recall - Measures the retriever’s ability to retrieve all necessary information required to answer the question.
https://docs.ragas.io/en/stable/concepts/metrics/context_recall.html

Context entities recall 0.2815270920302487
Context entities recall - Measure what fraction of entities are recalled from ground_truths. This metric is useful in fact-based use cases like tourism help desk, historical QA, etc.
https://docs.ragas.io/en/stable/concepts/metrics/context_entities_recall.html


### Evaluate LLM module


#### Run LLM on benchmarking dataset

Already done in Kaggle

#### Evaluate baseline

In [ ]:
benchmark_dataset = pd.read_csv("gdrive/MyDrive/LLM01 - RắnGPT/Code/Benchmark QA/llm_answer_to_benchmark_question.csv")
for i in range(len(benchmark_dataset)):
  benchmark_dataset.loc[i, 'contexts'] = eval(benchmark_dataset.loc[i, 'contexts'].replace('\n', ","))

In [ ]:
# len(benchmark_dataset['contexts'][0])

5

In [ ]:
dataset = Dataset.from_pandas(benchmark_dataset)

score = evaluate(dataset, metrics=[answer_relevancy, faithfulness, answer_correctness], llm=langchain_llm, embeddings=embeddings)
generator_score = score.to_pandas()

Evaluating:   0%|          | 0/174 [00:00<?, ?it/s]

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
domain: "googleapis.com"
metadata {
  key: "consumer"
  value: "projects/813003764678"
}
metadata {
  key: "quota_limit"
  value: "GenerateContentRequestsPerMinutePerProjectPerRegion"
}
metadata {
  key: "quota_limit_value"
  value: "60"
}
metadata {
  key: "quota_location"
  value: "us-central2"
}
metadata {
  key: "quota_metric"
  value: "generativelanguage.googleapis.com/generate_content_requests"
}
metadata {
  key: "service"
  value: "generativelanguage.googleapis.com"
}
, links {
  description: "Request a higher quota limit."
  url: "https://cloud.google.com/docs/quota#requesting_higher_quota"
}
].
domain: "googleapis.com"
metadata {
  key: "consumer"
  value: "projects/813003764678"
}
metadata {
  key: "quota_limit"
  value: "GenerateContentRequestsPerMinutePerProjectPerRegion"
}
metadata {
  key: "quota_limit_value"
  value: "60"
}
metadata {
  key: "quota_location"
  value: "us-central2"
}
metadata {
  key: "quota_metric

In [ ]:
print("Answer relevancy", generator_score['answer_relevancy'].mean())
print("Answer relevancy - Measures how relevant the answer is to the question")
print("https://docs.ragas.io/en/stable/concepts/metrics/answer_relevance.html")
print()
print("Faithfulness", generator_score['faithfulness'].mean())
print("Faithfulness - Measures the factual consistency of the answer to the context based on the question.")
print("https://docs.ragas.io/en/stable/concepts/metrics/faithfulness.html")
print()
print("Answer correctness", generator_score['answer_correctness'].mean())
print("Answer correctness - Measure semantic similarity between the generated answer and the ground truth, as well as factual similarity.")
print("https://docs.ragas.io/en/stable/concepts/metrics/answer_correctness.html")

Answer relevancy 0.5226939993576497
Answer relevancy - Measures how relevant the answer is to the question
https://docs.ragas.io/en/stable/concepts/metrics/answer_relevance.html

Faithfulness 0.8778787878787878
Faithfulness - Measures the factual consistency of the answer to the context based on the question.
https://docs.ragas.io/en/stable/concepts/metrics/faithfulness.html

Answer correctness 0.6635268504336559
Answer correctness - Measure semantic similarity between the generated answer and the ground truth, as well as factual similarity.
https://docs.ragas.io/en/stable/concepts/metrics/answer_correctness.html


In [ ]:
generator_score.to_csv("gdrive/MyDrive/LLM01 - RắnGPT/Code/Benchmark QA/llm_evaluation.csv", index=False)
generator_score.head()

question  \
0           Tên khoa học của rắn lục đuôi đỏ mắt nhỏ là gì?   
1                                 Rắn lột da bằng cách nào?   
2  Rắn lục đầu trắng phân bố ở những tỉnh nào của Việt Nam?   
3         Loài rắn biển này có bao nhiêu hàng vảy quanh cổ?   
4          Rắn sọc dưa có习 tính săn đuổi mồi chủ yếu là gì?   

                                            ground_truth  \
0                                  Cryptelytrops macrops   
1  Rắn lột da bằng cách lộn ngược cơ thể từ bên trong ra   
2                 Cao Bằng, Bắc Kạn, Vĩnh Phúc, Lạng Sơn   
3                                             31–43 hàng   
4            Chuột, ngoài ra có cả thằn lằn và ếch nhái.   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

#### Evaluate finetuned

In [ ]:
benchmark_dataset = pd.read_csv("gdrive/MyDrive/LLM01 - RắnGPT/Code/Benchmark QA/ft_llm_answer_to_benchmark_question.csv")
for i in range(len(benchmark_dataset)):
  benchmark_dataset.loc[i, 'contexts'] = eval(benchmark_dataset.loc[i, 'contexts'].replace('\n', ","))

In [ ]:
# len(benchmark_dataset['contexts'][0])

5

In [ ]:
dataset = Dataset.from_pandas(benchmark_dataset)

score = evaluate(dataset, metrics=[answer_relevancy, faithfulness, answer_correctness], llm=langchain_llm, embeddings=embeddings)
generator_score = score.to_pandas()

Evaluating:   0%|          | 0/174 [00:00<?, ?it/s]

In [ ]:
print("Answer relevancy", generator_score['answer_relevancy'].mean())
print("Answer relevancy - Measures how relevant the answer is to the question")
print("https://docs.ragas.io/en/stable/concepts/metrics/answer_relevance.html")
print()
print("Faithfulness", generator_score['faithfulness'].mean())
print("Faithfulness - Measures the factual consistency of the answer to the context based on the question.")
print("https://docs.ragas.io/en/stable/concepts/metrics/faithfulness.html")
print()
print("Answer correctness", generator_score['answer_correctness'].mean())
print("Answer correctness - Measure semantic similarity between the generated answer and the ground truth, as well as factual similarity.")
print("https://docs.ragas.io/en/stable/concepts/metrics/answer_correctness.html")

Answer relevancy 0.5693085359055676
Answer relevancy - Measures how relevant the answer is to the question
https://docs.ragas.io/en/stable/concepts/metrics/answer_relevance.html

Faithfulness 0.7160493827160493
Faithfulness - Measures the factual consistency of the answer to the context based on the question.
https://docs.ragas.io/en/stable/concepts/metrics/faithfulness.html

Answer correctness 0.6386810444930311
Answer correctness - Measure semantic similarity between the generated answer and the ground truth, as well as factual similarity.
https://docs.ragas.io/en/stable/concepts/metrics/answer_correctness.html


In [ ]:
generator_score.to_csv("gdrive/MyDrive/LLM01 - RắnGPT/Code/Benchmark QA/ft_llm_evaluation.csv", index=False)
generator_score.head()

question  \
0           Tên khoa học của rắn lục đuôi đỏ mắt nhỏ là gì?   
1                                 Rắn lột da bằng cách nào?   
2  Rắn lục đầu trắng phân bố ở những tỉnh nào của Việt Nam?   
3         Loài rắn biển này có bao nhiêu hàng vảy quanh cổ?   
4          Rắn sọc dưa có习 tính săn đuổi mồi chủ yếu là gì?   

                                            ground_truth  \
0                                  Cryptelytrops macrops   
1  Rắn lột da bằng cách lộn ngược cơ thể từ bên trong ra   
2                 Cao Bằng, Bắc Kạn, Vĩnh Phúc, Lạng Sơn   
3                                             31–43 hàng   
4            Chuột, ngoài ra có cả thằn lằn và ếch nhái.   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               